In [1]:
import pandas as pd
import numpy as np

import csv
import time

import datetime
import requests
import json
from time import sleep
import csv
import threading
import os

In [3]:
# Prometheus API endpoint
prometheus_api = "http://192.168.49.2:31090/api/v1/query"
query1 = """sum (container_memory_working_set_bytes{namespace="sock-shop"})"""
query2= "sum (machine_memory_bytes{})"

# CSV file path
csv_file_path = "Sock-shop_memory_usage.csv"
csv_header = ["Timestamp", "Value"]

def fetch_data():
    while True:
        # Fetch data from Prometheus
        params= {"query": query1}
        response1 = requests.get(prometheus_api, params=params)
        data1 = response1.json()

        params = {"query": query2}
        response2 = requests.get(prometheus_api, params=params)
        data2 = response2.json()
        
        total_machine_memory = data2['data']['result'][0]['value'][1]
        
        # Open CSV file in append mode
        with open(csv_file_path, mode="a", newline="") as csv_file:
            writer = csv.writer(csv_file)

            # Write header if file is empty
            if csv_file.tell() == 0:
                writer.writerow(csv_header)

            # Write data to CSV
            for result in data1["data"]["result"]:
                time = result["value"][0]
                dt_object = datetime.datetime.fromtimestamp(time)
                timestamp = dt_object.strftime('%Y-%m-%d %H:%M:%S')
                value = round(((float(result["value"][1]))/float(total_machine_memory))*100, 2)
                writer.writerow([timestamp, value])
        
        # Sleep for 5 seconds
        sleep(5)
fetch_data()

KeyboardInterrupt: 